In [1]:
#imports
import requests
import pandas as pd
import json
import time

In [2]:
class Release_Node:
    """
    Represents a node in a networkx diagram.with basic attributes and functionalities(TBD).

    Attributes:
        release_name (str): Name of the music release.
        release_mbid (str): MusicBrainz ID.
        release_credits (list): List of credits on the music release.
    """
    def __init__(self, release_name, release_data, release_credits):
        """
        Initializes the Release_Node object.

        Args:
            release_name (str): Name of the music release.
            release_mbid (str): MusicBrainz ID.
            release_credits (list): List of credits on the music release.
        """
        self.release_name = name
        self.release_mbid = data
        self.release_credits = []

In [3]:
def search_recording(artist_name, release_title):
    """
    Retrieves all release groups for a given artist using their MusicBrainz ID.

    Args:
        artist_name (str): Name of the artist.
        release_title (str): Name of the Release

    Returns:
        list: The MBID for the release. This is the initial node for the network.
    """
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][0]['id']
    return None

# Example usage:
artist = "Steely Dan"
album = "Katy lied"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

Recording MBID for 'Katy lied' by 'Steely Dan': a23e8312-b89e-4822-b94e-309ea95806d8


In [4]:
async def get_credits_for_recording(release_mbid):
    """
    Retrieves all the credits for a given release using the releases MusicBrainz ID.

    Args:
        release_mbid (str): MusicBrainz ID for the release.

    Returns:
        list: A list of all credited personnel (including artist) on the release based on the provided release mbid.
        This list includes the name of the credit, the role of the credit, and the mbid for the credit.
    """
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found'), credit.get('artist', {}).get('id', 'Not Found')))
    if data['relations']:
       for credit in data['relations']:
           trackNum = 'Not Found'
           trackTitle = 'Not Found'
           if credit['attributes'] == [] or not credit['attributes']:
               attributes = 'Not Found'
           else:
               attributes = credit['attributes']
           credits.append((trackNum, trackTitle, credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))

    creditNames = []
    creditList = []
    for credit in credits:
        if credit[2] not in creditNames:
            creditNames.append(credit[2])
            creditList.append((credit[2],credit[3], credit[4]))
    print(creditNames)
    print("\n")
    print(creditList)

    artist_mbid = creditList[0][2]
    print(artist_mbid)
    
    return credits

In [5]:
#Example usage:
if release_mbid:
    albumCredits = await get_credits_for_recording(release_mbid)
    #print(albumCredits)
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

time.sleep(2)

# Create the DataFrame
df = pd.DataFrame(albumCredits, columns = ['Track Number', 'Track Title', 'Artist', 'Role', 'Artist_MBID'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Display the DataFrame
df

['Walter Becker', 'Donald Fagen', 'Roger Nichols', 'Victor Feldman', 'Michael Omartian', 'David Paich', 'Jeff Porcaro', 'Chuck Rainey', 'Gary Katz', 'Not Found', 'Steely Dan', 'Wilton Felder', 'Hugh McCracken', 'Michael McDonald', 'Dean Parks', 'Larry Carlton', 'Phil Woods', 'Carolyn Willis', 'Denny Dias', 'Rick Derringer', 'Hal Blaine', 'Bill Perkins', 'Elliott Randall', 'Jimmie Haskell', 'Dorothy White']


[('Walter Becker', 'Not Found', '8cc72457-488e-44e6-a5b3-d2da843b7245'), ('Donald Fagen', 'Not Found', '70047e57-0153-4117-b0fc-a1d2e322e5ef'), ('Roger Nichols', 'Not Found', 'f706ae5b-de2c-4e4a-adc2-a1b9da839d70'), ('Victor Feldman', ['percussion'], 'c648f7ba-cee7-4f04-aaec-03299409682c'), ('Michael Omartian', ['keyboard'], 'b299bac7-80af-4266-8811-6297a16c8a37'), ('David Paich', ['keyboard'], '7cb00709-0f07-4a90-bcbe-f7ad5d57d8fd'), ('Jeff Porcaro', ['drums (drum set)'], '1658d3b1-0c07-41cd-a358-7bbea34903b2'), ('Chuck Rainey', ['bass'], '910504cf-866e-4ee6-90ca-d71da15c18ce'), (

,Track Number,Track Title,Artist,Role,Artist_MBID
0,A1,Black Friday,Walter Becker,Not Found,8cc72457-488e-44e6-a5b3-d2da843b7245
1,A1,Black Friday,Donald Fagen,Not Found,70047e57-0153-4117-b0fc-a1d2e322e5ef
2,A1,Black Friday,Roger Nichols,Not Found,f706ae5b-de2c-4e4a-adc2-a1b9da839d70
3,A1,Black Friday,Walter Becker,"[bass, guitar]",8cc72457-488e-44e6-a5b3-d2da843b7245
4,A1,Black Friday,Walter Becker,"[guitar, solo]",8cc72457-488e-44e6-a5b3-d2da843b7245
5,A1,Black Friday,Donald Fagen,[piano],70047e57-0153-4117-b0fc-a1d2e322e5ef
6,A1,Black Friday,Donald Fagen,[keyboard],70047e57-0153-4117-b0fc-a1d2e322e5ef
7,A1,Black Friday,Victor Feldman,[percussion],c648f7ba-cee7-4f04-aaec-03299409682c
8,A1,Black Friday,Victor Feldman,[vibraphone],c648f7ba-cee7-4f04-aaec-03299409682c
9,A1,Black Friday,Michael Omartian,[keyboard],b299bac7-80af-4266-8811-6297a16c8a37
